In [ ]:
import sys
import os
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', None)

# 현재 노트북 파일의 경로를 기준으로 프로젝트 루트 경로를 계산
project_root = os.path.abspath(os.path.join(os.getcwd(), '..', '..', '..', '..'))

# sys.path에 프로젝트 루트 경로가 없으면 추가
if project_root not in sys.path:
  sys.path.append(project_root)


In [2]:
from src.data.db_handler import DBHandler

db_handler = DBHandler(db_name="data_lake")

df = db_handler.fetch_data(table_name="kr_stock_balance_sheet")

column_index = df.columns

df.info()
df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5759 entries, 0 to 5758
Data columns (total 14 columns):
 #   Column      Non-Null Count  Dtype              
---  ------      --------------  -----              
 0   id          5759 non-null   int64              
 1   ticker      5759 non-null   object             
 2   stac_yymm   5759 non-null   object             
 3   cras        5759 non-null   object             
 4   fxas        5759 non-null   object             
 5   total_aset  5759 non-null   object             
 6   flow_lblt   5759 non-null   object             
 7   fix_lblt    5759 non-null   object             
 8   total_lblt  5759 non-null   object             
 9   cpfn        5759 non-null   object             
 10  cfp_surp    5759 non-null   object             
 11  prfi_surp   5759 non-null   object             
 12  total_cptl  5759 non-null   object             
 13  updated_at  5759 non-null   datetime64[ns, UTC]
dtypes: datetime64[ns, UTC](1), int64(1), obj

,id,ticker,stac_yymm,cras,fxas,total_aset,flow_lblt,fix_lblt,total_lblt,cpfn,cfp_surp,prfi_surp,total_cptl,updated_at
0,1,005930,202506,2121607.00,2927145.00,5048752.00,844026.00,209106.00,1053132.00,8975,99.99,99.99,3995620.00,2025-09-16 12:59:08.442450+00:00
1,2,005930,202503,2226857.00,2936910.00,5163767.00,902797.00,194828.00,1097625.00,8975,99.99,99.99,4066143.00,2025-09-16 12:59:08.442450+00:00
2,3,005930,202412,2270623.00,2874697.00,5145319.00,933263.00,190136.00,1123399.00,8975,99.99,99.99,4021921.00,2025-09-16 12:59:08.442450+00:00
3,4,005930,202409,2192351.00,2720722.00,4913073.00,870319.00,179941.00,1050260.00,8975,99.99,99.99,3862814.00,2025-09-16 12:59:08.442450+00:00
4,5,005930,202406,2178581.00,2678996.00,4857577.00,843549.00,178761.00,1022310.00,8975,99.99,99.99,3835267.00,2025-09-16 12:59:08.442450+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5754,5755,002710,201903,1567.00,1559.00,3127.00,972.00,1035.00,2006.00,223,99.99,99.99,1120.00,2025-09-16 12:59:08.442450+00:00
5755,5756,002710,201812,1522.00,1526.00,3048.00,932.00,994.00,1925.00,223,99.99,99.99,1122.00,2025-09-16 12:59:08.442450+00:00
5756,5757,002710,201809,1441.00,1497.00,2938.00,1644.00,224.00,1869.00,223,99.99,99.99,1069.00,2025-09-16 12:59:08.442450+00:00
5757,5758,002710,201806,1394.00,1513.00,2907.00,1645.00,219.00,1863.00,223,99.99,99.99,1044.00,2025-09-16 12:59:08.442450+00:00


In [3]:
final_cols = column_index[1:]

In [4]:
for col in final_cols:
  df[col] .replace(r'^\s*$', np.nan, regex=True, inplace=True)
  df[col].replace(['0', 0], np.nan, inplace=True)

missing_value_counts = df[final_cols].isnull().sum()

missing_value_counts

ticker         0
stac_yymm      0
cras           0
fxas           0
total_aset     0
flow_lblt      0
fix_lblt       0
total_lblt     0
cpfn          64
cfp_surp       0
prfi_surp      0
total_cptl     0
updated_at     0
dtype: int64

In [5]:
missing_cols = missing_value_counts[missing_value_counts >= 100].index.to_list()

final_cols = [col for col in final_cols if col not in missing_cols]

print(f"결측 컬럼 : {missing_cols}")

print(f"[원본 컬럼 : {len(column_index)-1}] - [결측 컬럼 : {len(missing_cols)}] = [후보 컬럼 : {len(final_cols)}]")

결측 컬럼 : []
[원본 컬럼 : 13] - [결측 컬럼 : 0] = [후보 컬럼 : 13]


In [6]:
nunique_counts = df[final_cols].nunique()

unique_cols = nunique_counts[nunique_counts == 1].index.to_list()

final_cols = [col for col in final_cols if col not in unique_cols]

print(f"유니크 컬럼 : {unique_cols}")

print(f"[원본 컬럼 : {len(column_index)-1}] - [결측 컬럼 : {len(missing_cols)}] - [유니크 컬럼 : {len(unique_cols)}] = [후보 컬럼 : {len(final_cols)}]")

유니크 컬럼 : ['cfp_surp', 'prfi_surp', 'updated_at']
[원본 컬럼 : 13] - [결측 컬럼 : 0] - [유니크 컬럼 : 3] = [후보 컬럼 : 10]


In [7]:
df[final_cols]

,ticker,stac_yymm,cras,fxas,total_aset,flow_lblt,fix_lblt,total_lblt,cpfn,total_cptl
0,005930,202506,2121607.00,2927145.00,5048752.00,844026.00,209106.00,1053132.00,8975,3995620.00
1,005930,202503,2226857.00,2936910.00,5163767.00,902797.00,194828.00,1097625.00,8975,4066143.00
2,005930,202412,2270623.00,2874697.00,5145319.00,933263.00,190136.00,1123399.00,8975,4021921.00
3,005930,202409,2192351.00,2720722.00,4913073.00,870319.00,179941.00,1050260.00,8975,3862814.00
4,005930,202406,2178581.00,2678996.00,4857577.00,843549.00,178761.00,1022310.00,8975,3835267.00
...,...,...,...,...,...,...,...,...,...,...
5754,002710,201903,1567.00,1559.00,3127.00,972.00,1035.00,2006.00,223,1120.00
5755,002710,201812,1522.00,1526.00,3048.00,932.00,994.00,1925.00,223,1122.00
5756,002710,201809,1441.00,1497.00,2938.00,1644.00,224.00,1869.00,223,1069.00
5757,002710,201806,1394.00,1513.00,2907.00,1645.00,219.00,1863.00,223,1044.00


| 컬럼명 | 한글컬럼명 | 중요도 | 이유 |
| :--- | :--- | :--- | :--- |
| `ticker` | 티커(종목 코드) | **필수** | 모든 금융 데이터(주가, 재무, 뉴스)를 연결하는 유일한 식별자(Identifier)입니다. 이 컬럼 없이는 분석 자체가 불가능합니다. |
| `stac_yymm` | 결산년월 | **필수** | 재무 데이터가 어느 시점의 데이터인지 명시하는 기준점으로, 시계열 분석 및 특정 시점의 뉴스 데이터와 결합할 때 반드시 필요합니다. |
| `cras` | 유동자산 | **상** | 기업의 단기 지급 능력을 나타내는 핵심 지표로, 유동성 위기 관련 뉴스 발생 시 변동성에 직접적인 영향을 줄 수 있는 주요 피처입니다. |
| `fxas` | 비유동자산 | **중** | 기업의 장기적인 성장성 및 안정성과 관련된 지표입니다. 단기 변동성보다는 장기적인 기업 가치 평가에 더 큰 영향을 줍니다. |
| `total_aset` | 자산총계 | **상** | 기업의 전체 규모를 나타내는 대표적인 지표이며, 부채 비율 등 핵심적인 재무 안정성 비율을 계산하는 데 필수적인 요소입니다. |
| `flow_lblt` | 유동부채 | **상** | 단기 부채 상환 압박을 나타내며, 유동자산과 함께 유동성 리스크를 측정하는 데 사용됩니다. 재무 리스크 관련 이벤트 분석에 중요합니다. |
| `fix_lblt` | 비유동부채 | **중** | 장기적인 재무 구조의 안정성을 평가하는 데 사용됩니다. 단기적인 시장 충격보다는 기업의 장기 신용도에 더 연관성이 높습니다. |
| `total_lblt` | 부채총계 | **상** | 기업의 총 부채 규모를 나타내며, 레버리지 수준과 재무 리스크를 평가하는 가장 기본적인 척도입니다. 변동성 예측에 중요한 요소입니다. |
| `cpfn` | 자본금 | **하** | 회계상 자본의 일부이지만, 기업의 실제 가치나 재무 상태를 직접적으로 반영하기보다는 상징적인 의미가 강해 피처로서의 중요도는 낮습니다. |
| `total_cptl` | 자본총계 | **상** | 자산총계에서 부채총계를 뺀 순자산으로, 기업의 재무 건전성과 안정성을 직접적으로 보여주는 핵심 지표입니다. |

In [8]:
final_cols = [
  'ticker',
  'stac_yymm',
  'cras',
  'fxas',
  'total_aset',
  'flow_lblt',
  'fix_lblt',
  'total_lblt',
  'total_cptl'
]

In [9]:
df[final_cols]

,ticker,stac_yymm,cras,fxas,total_aset,flow_lblt,fix_lblt,total_lblt,total_cptl
0,005930,202506,2121607.00,2927145.00,5048752.00,844026.00,209106.00,1053132.00,3995620.00
1,005930,202503,2226857.00,2936910.00,5163767.00,902797.00,194828.00,1097625.00,4066143.00
2,005930,202412,2270623.00,2874697.00,5145319.00,933263.00,190136.00,1123399.00,4021921.00
3,005930,202409,2192351.00,2720722.00,4913073.00,870319.00,179941.00,1050260.00,3862814.00
4,005930,202406,2178581.00,2678996.00,4857577.00,843549.00,178761.00,1022310.00,3835267.00
...,...,...,...,...,...,...,...,...,...
5754,002710,201903,1567.00,1559.00,3127.00,972.00,1035.00,2006.00,1120.00
5755,002710,201812,1522.00,1526.00,3048.00,932.00,994.00,1925.00,1122.00
5756,002710,201809,1441.00,1497.00,2938.00,1644.00,224.00,1869.00,1069.00
5757,002710,201806,1394.00,1513.00,2907.00,1645.00,219.00,1863.00,1044.00


In [10]:
final_df = df[final_cols].copy()

# --- 1단계: 날짜 타입 자동 변환 ---
remaining_object_cols = final_df.select_dtypes(include='object').columns.tolist()
for col in remaining_object_cols:
  original_valid_count = final_df[col].count()
  if original_valid_count == 0: continue
  
  for format in ['%Y%m', '%Y%m%d']:
    date_series = pd.to_datetime(final_df[col], format=format,errors='coerce')
    success_rate = date_series.count() / original_valid_count
    
    if success_rate > 0.95: # 95% 임계값
      final_df[col] = date_series
      break

# --- 2단계: 저비율 카테고리 자동 변환 ---
object_cols = final_df.select_dtypes(include='object').columns.tolist()
for col in object_cols:
  ratio = final_df[col].nunique() / len(final_df[col])
  if ratio < 0.05: # 5% 임계값
    final_df[col] = final_df[col].astype('category')


In [11]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5759 entries, 0 to 5758
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   ticker      5759 non-null   category      
 1   stac_yymm   5759 non-null   datetime64[ns]
 2   cras        5759 non-null   object        
 3   fxas        5759 non-null   object        
 4   total_aset  5759 non-null   object        
 5   flow_lblt   5759 non-null   object        
 6   fix_lblt    5759 non-null   object        
 7   total_lblt  5759 non-null   object        
 8   total_cptl  5759 non-null   object        
dtypes: category(1), datetime64[ns](1), object(7)
memory usage: 381.0+ KB


| 컬럼명 | 한글컬럼명 | 현재 타입 (Pandas) | 적정 타입 (Pandas) | 이유 |
| :--- | :--- | :--- | :--- | :--- |
| `ticker` | 티커(종목 코드) | `object` | `category` | 종목 코드는 반복적으로 나타나는 문자열이므로, **`category`** 타입으로 변환 시 **메모리 사용량을 크게 최적화**할 수 있으며 그룹화(groupby)나 조인(join) 연산 시 **속도 향상**을 기대할 수 있습니다. |
| `stac_yymm` | 결산년월 | `object` | `datetime64[ns]` | 날짜/시간 관련 데이터는 **`datetime64`** 타입으로 관리해야 **시계열 분석, 시간 기반 필터링 및 집계** 등 Pandas의 강력한 시간 관련 기능을 온전히 활용할 수 있습니다. `pd.to_datetime` 함수를 사용한 변환이 필요합니다. |
| `cras` | 유동자산 | `object` | `int64` | 금액 데이터는 정확한 수치 연산이 필수적이므로 **`int64`** 타입으로 변환해야 합니다. 이는 **메모리 효율성**과 **계산 성능**을 모두 확보하는 표준적인 방법입니다. |
| `fxas` | 비유동자산 | `object` | `int64` | 유동자산과 마찬가지로, 재무 비율 등을 계산하기 위해 **빠른 수치 연산을 지원**하는 **`int64`** 타입이 적합합니다. |
| `total_aset` | 자산총계 | `object` | `int64` | 조 단위의 큰 금액도 표현 가능하며, 다른 재무 컬럼들과의 일관된 타입으로 **데이터 정합성**을 유지하고 **효율적인 연산**을 지원합니다. |
| `flow_lblt` | 유동부채 | `object` | `int64` | `object` (문자열) 타입 대비 **메모리를 훨씬 효율적으로 사용**하며, 재무 분석에 필요한 **사칙연산을 비롯한 다양한 수학적 연산**이 가능해집니다. |
| `fix_lblt` | 비유동부채 | `object` | `int64` | 다른 재무 수치들과 타입을 일치시켜, 데이터프레임 내에서 **일관된 처리 및 분석**이 가능하도록 **`int64`**로 변환하는 것이 바람직합니다. |
| `total_lblt` | 부채총계 | `object` | `int64` | **`int64`**는 부채와 관련된 모든 **통계적 계산(합계, 평균 등)의 성능을 보장**하며, 분석 과정에서의 오류 발생 가능성을 줄여줍니다. |
| `total_cptl` | 자본총계 | `object` | `int64` | 자산, 부채와 함께 기업의 재무 상태표를 구성하는 핵심 요소이므로, 정확한 계산을 위해 **`int64`**로 변환하는 것이 필수적입니다. |

In [12]:
dtype_map = {
  'Int64': ['cras', 'fxas', 'total_aset', 'flow_lblt', 'fix_lblt', 'total_lblt', 'total_cptl'],
  'datetime64[ns]': ['stac_yymm'],
  'category': ['ticker']
}

for dtype, cols in dtype_map.items():
  # DataFrame에 실제 존재하는 컬럼만 필터링
  valid_cols = [col for col in cols if col in final_df.columns]
  if not valid_cols:
    continue

  try:
    if dtype == 'Int64':
      for col in valid_cols:
        numeric_col = pd.to_numeric(final_df[col], errors='coerce')
        if not numeric_col.isnull().all():
          final_df[col] = numeric_col.astype('Int64')
    
    elif dtype == 'float64':
      final_df[valid_cols] = final_df[valid_cols].apply(pd.to_numeric, errors='coerce')

    elif dtype == 'datetime64':
      for col in valid_cols:
        final_df[col] = pd.to_datetime(final_df[col], errors='coerce')
    
    else: # 'category' 등 .astype()으로 처리 가능한 나머지 타입
      final_df[valid_cols] = final_df[valid_cols].astype(dtype)

  except (ValueError, TypeError) as e:
    print(f"⚠️ 경고: 컬럼 {valid_cols}을(를) '{dtype}'으로 변환 중 오류. 건너뜁니다. (에러: {e})")


In [13]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5759 entries, 0 to 5758
Data columns (total 9 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   ticker      5759 non-null   category      
 1   stac_yymm   5759 non-null   datetime64[ns]
 2   cras        5759 non-null   Int64         
 3   fxas        5759 non-null   Int64         
 4   total_aset  5759 non-null   Int64         
 5   flow_lblt   5759 non-null   Int64         
 6   fix_lblt    5759 non-null   Int64         
 7   total_lblt  5759 non-null   Int64         
 8   total_cptl  5759 non-null   Int64         
dtypes: Int64(7), category(1), datetime64[ns](1)
memory usage: 420.3 KB
